#Important datas


In [ ]:
import sys
import numpy as np
from scipy import ndimage as ndi
from skimage import io, filters, color, transform
import cv2 as cv
from scipy.ndimage.morphology import distance_transform_edt
from skimage.morphology import erosion, disk
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

pages = ["dasrtseldesjdisc00frit_0007", "dasrtseldesjdisc00frit_0008", "dasrtseldesjdisc00frit_0009", "dasrtseldesjdisc00frit_0010", "dasrtseldesjdisc00frit_0011"]

#TODO: Poniżej należy podać ścieżkę do głównego folderu
folder_path = "FolderGlowny"
columns=['Char', 'Image']
sizes = range(4, 71)

In [ ]:
# module level variables ##########################################################################
#Size of the digit
#max numbers of one char 
max_size = 100
#enable chars
enable_chars = ['A', 'a', 'B', 'b', 'C', 'c', 'D', 'd', 'E', 'e', 'F', 'f', 'G', 'g', 'H', 'h', 'I', 'i', 'J', 'j', 'K', 'k', 'L', 'l', 'M', 'm', 'N', 'n', 'O', 'o', 'P', 'p',
                'R', 'r', 'S', 's', 'T', 't', 'U', 'u', 'W', 'w', 'Y', 'y', 'Z', 'z', 'Ä', 'ä', 'Ö', 'ö', 'Q', 'q', 'ẞ', 'ß', 'Ü', 'ü', 'V', 'v', 'X', 'x', '1','2','3', '4', 
                '5', '6', '7', '8', '9', '0', ',', '.', '(', ')', '-', '\'', '\"','+', '=','#', '^', '*', '?', '[', ']', '%', ':',';',
                'ch', 'ck', 'st', 'll', 'ss', 'fi', 'si', 'ff', 'fl']
x = {char: [] for char in enable_chars}
###################################################################################################

In [ ]:
def save_to_npz_file(x_train, x_test, y_train, y_test, size):

    result_file_path = folder_path+ '/TrainingFiles/Final/' + str(size) + '.npz'
    np.savez_compressed(result_file_path,
                    x_train = x_train,x_test = x_test, y_train = y_train,  y_test = y_test)
    
def load_to_npz_file(size):
    NPZFile = np.load(folder_path+ '/TrainingFiles/Final/' + str(size) + '.npz', allow_pickle= True)
    x_train, y_train, x_test, y_test = [NPZFile[f] for f in ['x_train', 'y_train', 
                                    'x_test', 'y_test']]
    return x_train, x_test, y_train, y_test

In [ ]:
def shuffle(x_train, x_test, y_train, y_test, seed):
    x = np.vstack((x_train, x_test))
    y = np.hstack([y_train, y_test])
    new_x_train, new_x_test, new_y_train, new_y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=seed)
    return new_x_train, new_x_test, new_y_train, new_y_test

#Utworzenie finalnych zbiorów danych o określonym rozmiarze przestrzeni cech

In [ ]:
def make_final_data_train_file(X, result_size):
    # data_frame = pd.DataFrame(columns=['Vector'])
    result_list = []
    for x in X:
        result_size_image = cv.resize(x, dsize=(result_size, result_size), interpolation=cv.INTER_CUBIC)
        vector = result_size_image.reshape((1, (result_size*result_size)))

        result_list.append(vector[0])
        # data_frame = data_frame.append({'Vector': vector[0]}, ignore_index=True);
    return np.array(result_list)

In [ ]:
NPZFile = np.load(folder_path+ '/TrainingFiles/DataTrain(NotResized).npz', allow_pickle= True)
X_train, Y_train, X_test, Y_test = [NPZFile[f] for f in ['X_train', 'y_train', 
                                    'X_test', 'y_test']]

In [ ]:
y_test = Y_test
y_train = Y_train
for size in sizes:
    print(size)
    x_train = make_final_data_train_file(X_train, size)
    x_test = make_final_data_train_file(X_test, size)
    save_to_npz_file(x_train, x_test, y_train, y_test, size)

70


# Badania przestrzeni cech


In [ ]:
import time
import pandas as pd
import pickle
import sys

def TestClassifier(classifier, x_test, y_test):

    start = time.process_time()
    score = classifier.score(x_test, y_test)
    time_proces = time.process_time() - start
    data = {'Score': [score], 'TestTime': [time_proces]}
    resultDf = pd.DataFrame(data = data, columns =['Score', 'TestTime'])
    return resultDf
def TrainClassifier(classifier, x_train, y_train):

    start = time.process_time()
    classifier.fit(x_train, y_train)
    time_proces = time.process_time() - start   
    p = pickle.dumps(classifier)
    data = {'TrainingTime': [time_proces], 'ModelSize': [sys.getsizeof(p)]}
    resultDf = pd.DataFrame(data = data,columns =['TrainingTime', 'ModelSize'])
    return resultDf 

In [ ]:
def TestSingleClassifier(classifier, x_train, x_test, y_train, y_test):
    singleTest = pd.DataFrame(data = [str(classifier)], columns=['Clasifier'])
    result = TrainClassifier(classifier, x_train, y_train)
    singleTest = pd.concat([singleTest, result], axis=1)
    result = TestClassifier(classifier, x_test, y_test)
    singleTest = pd.concat([singleTest, result], axis=1)
    return singleTest

In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
classifiers = {KNeighborsClassifier(), SVC()}
scores = pd.DataFrame(columns = ["Score", "TestTime", 'TrainingTime', 'ModelSize', 'Clasifier', 'Seed', 'Size'])
seeds = {1,2,3,2077, 4,5,6,7,8, 9}

for size in sizes:
    print(size)
    x_train, x_test, y_train, y_test = load_to_npz_file(size)
    stdSC = StandardScaler()
    x_train = stdSC.fit_transform(x_train)
    x_test = stdSC.fit_transform(x_test)
    seedsTest = pd.DataFrame(columns = ["Score", "TestTime", 'ModelSize', 'TrainingTime', 'Clasifier', 'Seed'])
    for seed in seeds:
        x_train, x_test, y_train, y_test = shuffle(x_train, x_test, y_train, y_test, seed)
        seedTest = pd.DataFrame(columns = ["Score", "TestTime", 'ModelSize', 'TrainingTime', 'Clasifier'])
        for classifier in classifiers:
            singleTest = TestSingleClassifier(classifier, x_train, x_test, y_train, y_test)
            seedTest = seedTest.append(singleTest)
        seedTest['Seed'] = seed
        seedsTest = seedsTest.append(seedTest)
    seedsTest['Size'] = size
    scores = scores.append(seedsTest)
    scores.to_csv(folder_path + "/Result/FeautreSizeTest3.csv")

67
68
69
70


#Badanie pojedynczego rozmiaru przestrzeni cech

##

In [ ]:
import time
def FindBestClassifier(classifierType, params, x_train, y_train):
    start = time.process_time()
    
    classifier = GridSearchCV(
        classifierType,
        params,
        verbose = 2,
        cv = 3,
        n_jobs = -4)
    classifier.fit(x_train, y_train)
    searchingTime = time.process_time() - start
    params = classifier.best_params_
    return params, searchingTime

def TestClassifier(classifierType, params, x_train, y_train, x_test, y_test):
    classifier = classifierType.set_params(**params)
    
    classifier.fit(x_train, y_train)
    score = classifier.score(x_test, y_test)
 
    y_pred = classifier.predict(x_test)
    matrix_of_errors = GetMatrixOfErrors(y_test, y_pred)

    return score, matrix_of_errors
def TestClassifier(classifier, x_train, y_train, x_test, y_test):
    classifier.fit(x_train, y_train)
    score = classifier.score(x_test, y_test)
 
    y_pred = classifier.predict(x_test)
    matrix_of_errors = GetMatrixOfErrors(y_test, y_pred)

    return score, matrix_of_errors

def GetMatrixOfErrors(y_test, y_pred):
    sorted_chars = np.unique(y_test)
    sorted_chars.sort()
    matrix_of_errors = pd.DataFrame(0, columns = sorted_chars, index = sorted_chars)
    for  (y_test, y_pred) in (zip(y_test, y_pred)):
        matrix_of_errors[y_test][y_pred] = matrix_of_errors[y_test][y_pred] + 1
    return matrix_of_errors

In [ ]:
scoreDf = pd.DataFrame(columns =['Size', 'Score', 'Params', 'SearchingTime', 'Classifier'])

In [ ]:

from sklearn.ensemble import RandomForestClassifier
classifierType =  RandomForestClassifier(n_jobs=-4, bootstrap=True)
params = {

    'n_estimators': [100, 200, 300, 500],
    'criterion' : ['entropy', 'gini']
    'max_depth': [80, 90, 100, 110, 120, 130, 200],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1 ],
    'max_features': ['sqrt', 'log2']
}


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

classifierType = KNeighborsClassifier()
params = {
    'n_neighbors': [3, 5, 7, 9, 11, 13],
    'weights' : ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 
    'metric': ['minkowski', 'cityblock', 'cosine', 'euclidean', 'haversine', 'l1', 'l2', 'manhattan', 'nan_euclidean']
}

In [ ]:
from sklearn.svm import SVC
classifierType = SVC()
params = {
        'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
        'gamma' : ['scale', 'auto'],
        'C':[0.1, 1, 2, 10, 20, 50]
    } 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifierType = DecisionTreeClassifier()
params = {
        'criterion' : ['gini','entropy', 'log_loss'],
        'splitter': ['best', 'random'],
        'max_depth': [10, 50, 100, 200, 500, 1000, None],
        'min_samples_split': [1, 2, 5, 10, 20, 50, 100],
        'min_samples_leaf': [1, 2, 5, 10, 20, 50, 100],
        'max_features': ['auto', 'sqrt', 'log2']
    }


In [ ]:

import pandas as pd
from sklearn.preprocessing import StandardScaler

sizes = {15}
classifierName = str(classifierType)
for size in sizes:
    print(size)
    x_train, x_test, y_train, y_test = load_to_npz_file(size)
    x, y = np.concatenate((x_train, x_test), axis=0), np.concatenate((y_train, y_test), axis=0)
    params, searchingTime = FindBestClassifier(classifierType, params, x, y)
    score, matrix_of_errors = TestClassifier(classifierType, params, x_train, y_train, x_test, y_test)
    matrix_of_errors.to_csv(folder_path + "/Result/MatrixOfErrors/" + str(classifierName) + str(size) + ".csv")

    scoreDf = scoreDf.append({'Size': size, 'Score': score, 'Params': params, 'SearchingTime': searchingTime, 'Classifier': classifierName}, ignore_index = True)



In [ ]:
scoreDf

In [ ]:
scoreDf.to_csv(folder_path + "/Result/FindBestParams.csv")